**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

In [80]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [81]:
import pandas as pd

In [83]:
stats = pd.read_csv('keywords.csv')

In [84]:
""" Функция определения региона по данным словаря geo_data """
def geo_class(row):
    for i,k in geo_data.items():
        for j in k:
            if j in row['keyword']:
                return i
    
    return 'undefined'

In [85]:
""" Добавляем стобец со значениями из функции geo_class """
stats['region']=stats.apply(geo_class,axis=1)

In [86]:
stats.region.value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [87]:
task2 = pd.read_csv('ratings.csv')
task2.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [88]:
"""
формирую новый датафрейм с уникальными movieId и вспомогательным столбцом
mean, в который записываю средние значения рейтинга
""" 
task2_1=task2.groupby('movieId').agg(['mean'])['rating']

In [89]:
""" Функция class_rating возвращает класс рейтинга в виде строки"""
def class_rating(row):
    if row['mean']<=2:
        return 'низкий рейтинг'
    elif row['mean']<=4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [90]:
""" Создаю столбец class со значениями функции class_rating """
task2_1['class']=task2_1.apply(class_rating,axis=1)

In [91]:
task2_1[['class']].loc[300:305]

,class
movieId,
300,средний рейтинг
301,высокий рейтинг
302,средний рейтинг
303,средний рейтинг
304,средний рейтинг
305,средний рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [92]:
task3=pd.read_csv('ratings.csv')
task3.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [93]:
"""
формирую новый датафрейм с уникальными userId и вспомогательными столбцами
min и max, в которых записываю минимальные и максимальные значения для каждого userId 
""" 
task3_1=task3.groupby('userId').agg(['min','max'])['timestamp']
task3_1.head()

,min,max
userId,,
1,1260759108,1260759205
2,835355395,835356246
3,1298861589,1298932787
4,949778714,949982274
5,1163373044,1163375145


In [94]:
""" Создаю столбец ratings со значениями, равными количеству поставленных оценок """
task3_1['ratings']=task3.groupby('userId').agg(['count'])['rating']
task3_1.head()

,min,max,ratings
userId,,,
1,1260759108,1260759205,20
2,835355395,835356246,76
3,1298861589,1298932787,51
4,949778714,949982274,204
5,1163373044,1163375145,100


In [95]:
""" Фильтрую значения в столбце ratings - оставляем только пользователей с >=100 оценками """
task3_1=task3_1[(task3_1['ratings']>=100)]

In [96]:
""" Функция вычисляет Lifetime"""
def lifetime(row):
    return row['max']-row['min']

In [97]:
""" создаю столбец lifetime, в который заношу значения функции lifetime

    ВОПРОС: эту простую операцию вычитания, наверное, можно без функции сделать?
    Если да, напишите, пожалуйста, как?, в комментарии к домашке   """

task3_1['lifetime']=task3_1.apply(lifetime,axis=1)

In [98]:
task3_1.head()

,min,max,ratings,lifetime
userId,,,,
4,949778714,949982274,204,203560
5,1163373044,1163375145,100,2101
8,1154389340,1154474527,116,85187
15,997937239,1469330735,1700,471393496
17,1127468587,1127476640,363,8053


In [99]:
from statistics import mean

In [100]:
""" Вычисляю среднее значение timestamp всех пользователей и перевожу в дни"""
mean(task3_1.lifetime)/3600/24

455.2285713719898

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

In [101]:
years=[]
for i in range(1950,2011):
    years.append(i)

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

In [102]:
""" функция не просто ищет год в поле title, а на всякий случай, если само название фильма тоже содержит год, 
    то во всей строке выберет последнее значение, похожее на год и подходящее по параметрам,
    как наиболее вероятно соответствующее именно году выпуска фильма"""
def production_year(row):
    check=False
    temp_year=[]
    temp_index=[]
    for i in years:
        if row['title'].find(str(i))!=-1:
            temp_year.append(years.index(i))
            temp_index.append(row['title'].find(str(i)))
            check=True
    if not check:
        return 1900
    else:
        max_index=temp_index.index(max(temp_index))
        return years[temp_year[max_index]]

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

In [103]:
data2=pd.read_csv('movies.csv')
data2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [104]:
data2['year']=data2.apply(production_year,axis=1)

In [105]:
data2.loc[621:624]

,movieId,title,genres,year
621,743,Spy Hard (1996),Comedy,1996
622,745,Wallace & Gromit: A Close Shave (1995),Animation|Children|Comedy,1995
623,746,Force of Evil (1948),Film-Noir,1900
624,747,"Stupids, The (1996)",Comedy,1996


4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [106]:
data1=pd.read_csv('ratings.csv')
data1.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [107]:
""" присоединяем таблицу с рейтингами фильмов"""
joined = data1.merge(data2, on='movieId', how='left')

In [108]:
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [111]:
"""
формирую новый датафрейм с уникальными годами year и столбцом
mean, в который записываю средние значения рейтинга для каждого года

""" 
joined2=joined.groupby('year').agg(['mean'])['rating']

In [112]:
""" сортируем средний рейтинг по убыванию

    судя по выводу первых 15 значений, гипотезу можно считать доказанной"""
joined2.sort_values('mean', ascending=False).head(15)

,mean
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1950,3.915254
1977,3.905786
